In [1]:
using CSV, LinearAlgebra, Statistics, DataFrames, Optim, ForwardDiff, Distributed, FixedEffectModels, Random
cd("C:\\Users\\Veli\\Downloads\\Code\\data\\clean")

In [2]:
versioninfo()

Julia Version 1.9.1
Commit 147bdf428c (2023-06-07 08:27 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 8 × Intel(R) Core(TM) i7-4770 CPU @ 3.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, haswell)
  Threads: 5 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 4


In [3]:
ENV["COLUMNS"] = 10000
ENV["LINES"] = 10000
ENV["LIMIT"] = 10000;

In [45]:
data = CSV.read("NH_MA_data_2020.csv", DataFrame)
df = data[data.DEASCHEDULE .== "CII",:]

filter!(row -> row.QTY > 0, df)
filter!(row -> row.AGE != "90+", df)
df.AGE = string.(df.AGE)
df.AGE = parse.(Int, df.AGE)
filter!(row -> row.NH_COUNTY_CODE != 999, df)
filter!(row -> row.MEMBER_COUNTY != 999, df)
filter!(row -> row.PROPRIETARYNAME != "ENDOCET", df)
filter!(row -> row.SV_STAT == "P", df)
filter!(row -> row.AMT_BILLED != 0.0, df)

df.PRICE = df.AMT_BILLED ./ df.QTY
prices = collect(df.PRICE)
product_ids = unique(df.PROPRIETARYNAME[1:length(prices)])
product_dict = Dict{String, Int64}()

for (index, value) in enumerate(product_ids)
    product_dict[value] = index
end

df.product_id = map(x -> product_dict[x], df.PROPRIETARYNAME);

In [46]:
prod_id_grped = groupby(df, :product_id);

In [47]:
product_ids_to_drop = []
for i in 1:length(prod_id_grped)
    length = size(prod_id_grped[i],1)
    product = prod_id_grped[i].product_id[1]
    if length <= 10 
        println("Product ", product, " has ", length, " claims.")
        push!(product_ids_to_drop, product)
    end
end

Product 27 has 4 claims.
Product 33 has 8 claims.
Product 38 has 9 claims.
Product 40 has 1 claims.
Product 46 has 1 claims.
Product 49 has 7 claims.
Product 51 has 8 claims.
Product 52 has 2 claims.
Product 55 has 6 claims.
Product 57 has 1 claims.
Product 58 has 3 claims.
Product 59 has 2 claims.
Product 61 has 1 claims.
Product 62 has 4 claims.
Product 63 has 8 claims.
Product 64 has 5 claims.
Product 65 has 1 claims.


In [48]:
filter!(row -> !(row.product_id in product_ids_to_drop), df);

In [49]:
avg_price_df = combine(groupby(df, :product_id), :PRICE => mean => :AVG_PRICE)
avg_price_dict = Dict(zip(avg_price_df.product_id, avg_price_df.AVG_PRICE))
df.OTHER_PRICES = zeros(size(df,1));

In [50]:
other_prices_list = []
for row in eachrow(df)
    new_dict = copy(avg_price_dict)
    new_dict[row.product_id] = row.PRICE
    price_vector = []
    for (prod_id,price) in enumerate(new_dict)
        push!(price_vector, price[2])
    end
    push!(other_prices_list, price_vector)
end

In [51]:
prices = collect(df.PRICE)
df.COINS_PERCENT = df.AMT_COINS ./ df.AMT_BILLED
coinsurance = collect(df.COINS_PERCENT)
df.FLAT_PAY = (df.AMT_COPAY + df.AMT_DEDUCT) ./ df.QTY
copay = collect(df.FLAT_PAY);

In [11]:
function choice_prob_numerator(α,δ_j,price,coinsurance,copay)
    util(α, p, δ_j) = exp.(α .* p .+ δ_j)
    p = price .* coinsurance .+ copay
    utility = util(α,p,δ_j)
    return utility
end

function choice_prob_denominator(α_scalar,δ_vector,price_vector,coinsurance_scalar,copay_scalar)
    util_values = choice_prob_numerator.(α_scalar, δ_vector, price_vector, coinsurance_scalar, copay_scalar)
    total = sum.(util_values)
    return total
end

function ll_fnc(params, prices, coinsurance, copay, df)
    α = params[end]
    δ = params[1:end-1]
    vector_δ = fill(δ,length(prices))
    
    numerator_list = choice_prob_numerator(α,δ[df.product_id],prices,coinsurance,copay)
    
    denominator_list = choice_prob_denominator(α,vector_δ,other_prices_list,coinsurance,copay)
        
    ll = sum(log.(numerator_list ./ (1 .+ denominator_list)))
    
    return -ll
end

ll_fnc (generic function with 1 method)

## Random Sampling Code

In [299]:
prod_id_grped = groupby(df, :product_id)
weights_dict = Dict{Int64, Float64}()
for i in 1:length(prod_id_grped)
    length = size(prod_id_grped[i],1)
    product = prod_id_grped[i].product_id[1]
    #println("Product ", product, " has ", length, " claims.")
    weights_dict[product] = length / size(df_21,1)
end

In [301]:
n = 5000
no_of_obs_dict = Dict{Int64, Int64}()
for (prod_id, weight) in enumerate(weights_dict)
    #println("Product ",weight[1], " : ", Int(round(weight[2]*n)))
    no_of_obs_dict[weight[1]] = Int(round(weight[2]*n))
end

In [302]:
column_names = names(df_21)
sampled_df = DataFrame([Symbol(name) => Vector{Any}() for name in column_names])

Row,PROV_KEY,PROV_CW_KEY,FACILITY_NAME,PROV_TYPE,PROV_TYPE_ORIG,PROV_CLINIC_STATE,NH_COUNTY_CODE,NDC,COVERAGE_CLASS,FROM_YEAR,CLAIM_ID_KEY,SV_LINE,SV_STAT,AGE,SEX,MEMBER_COUNTY,MEMBER_STATE,PHARMACY_COUNTY,INSURANCE_TYPE,BILL_PROV_CW_KEY,QTY,AMT_BILLED,AMT_PAID,AMT_DEDUCT,AMT_COINS,AMT_COPAY,NDC_PROD_NAME,BRAND_STATUS,RX_DAYS_SUPPLY,RX_INGR_COST,RX_DISP_FEE,RX_DAW,RX_REFILLS,COMPOUND,CLAIM_STATUS_ORIG,IMPUTED_SERVICE_KEY,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,NONPROPRIETARYNAME,DOSAGEFORMNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,DEASCHEDULE,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,DISTANCE,PRICE,product_id,COINS_PERCENT,FLAT_PAY
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any


In [303]:
for product_index in 1:size(prod_id_grped,1)
    prod_id = unique(prod_id_grped[product_index].product_id)[1]
    no_of_obs = no_of_obs_dict[prod_id]
    for obs in randperm(size(prod_id_grped[product_index],1))[1:no_of_obs]
        push!(sampled_df, prod_id_grped[product_index][obs,:])
    end
end

 Need to reset the product_id index and other price list to fix the indexing problem related to $\delta$.

In [348]:
product_ids = unique(sampled_df.PROPRIETARYNAME[1:length(sample_prices)])
product_dict = Dict{String, Int64}()

for (index, value) in enumerate(product_ids)
    product_dict[value] = index
end

sampled_df.product_id = map(x -> product_dict[x], sampled_df.PROPRIETARYNAME);
avg_price_df = combine(groupby(sampled_df, :product_id), :PRICE => mean => :AVG_PRICE)
avg_price_dict = Dict(zip(avg_price_df.product_id, avg_price_df.AVG_PRICE))

other_prices_list = []
for row in eachrow(sampled_df)
    new_dict = copy(avg_price_dict)
    new_dict[row.product_id] = row.PRICE
    price_vector = []
    for (prod_id,price) in enumerate(new_dict)
        push!(price_vector, price[2])
    end
    price_vector = convert(Vector{Float64}, price_vector)
    push!(other_prices_list, price_vector)
end

In [349]:
sample_prices = convert(Vector{Float64}, sampled_df.PRICE)
sample_coins = convert(Vector{Float64}, sampled_df.COINS_PERCENT);
sample_copay = convert(Vector{Float64}, sampled_df.FLAT_PAY);
α = 0.1
δ = zeros(length(unique(sampled_df.PROPRIETARYNAME)))
init_params = vcat(δ,α)
lower = fill(-100.0, length(init_params))
upper = fill(100.0, length(init_params));

In [342]:
@time ll_fnc(init_params,sample_prices,sample_coins,sample_copay,sampled_df)

  0.035038 seconds (1.00 M allocations: 20.631 MiB)


19343.180551568934

In [19]:
function optimize_function(init_params,prices,coinsurance,copay,df,lower,upper, fnc)
    res = optimize(params -> fnc(params, prices, coinsurance, copay,df), lower, upper, init_params)
    return res
end

optimize_function (generic function with 1 method)

In [352]:
@time res = optimize_function(init_params,sample_prices,sample_coins,sample_copay,sampled_df,lower,upper, ll_fnc)

608.659152 seconds (3.21 G allocations: 597.148 GiB, 13.77% gc time, 0.10% compilation time)


 * Status: success

 * Candidate solution
    Final objective value:     1.521490e+04

 * Found with
    Algorithm:     Fminbox with L-BFGS

 * Convergence measures
    |x - x'|               = 8.93e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.41e-11 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.28e-07 ≰ 1.0e-08

 * Work counters
    Seconds run:   609  (vs limit Inf)
    Iterations:    7
    f(x) calls:    1322
    ∇f(x) calls:   1322


In [356]:
δ_5000 = res.minimizer[1:end-1]
α_5000 = res.minimizer[end];

## Full Data Analysis

In [52]:
prices = convert(Vector{Float64}, df.PRICE)
coins = convert(Vector{Float64}, df.COINS_PERCENT);
copay = convert(Vector{Float64}, df.FLAT_PAY);
α = -0.1
δ = zeros(length(unique(df.PROPRIETARYNAME)))
init_params = vcat(δ,α)
lower = fill(-100.0, length(init_params))
upper = fill(100.0, length(init_params))

product_ids = unique(df.PROPRIETARYNAME[1:length(prices)])
product_dict = Dict{String, Int64}()

for (index, value) in enumerate(product_ids)
    product_dict[value] = index
end        

df.product_id = map(x -> product_dict[x], df.PROPRIETARYNAME);
avg_price_df = combine(groupby(df, :product_id), :PRICE => mean => :AVG_PRICE)
avg_price_dict = Dict(zip(avg_price_df.product_id, avg_price_df.AVG_PRICE))

other_prices_list = []
for row in eachrow(df)
    new_dict = copy(avg_price_dict)
    new_dict[row.product_id] = row.PRICE
    price_vector = []
    for (prod_id,price) in enumerate(new_dict)
        push!(price_vector, price[2])
    end
    price_vector = convert(Vector{Float64}, price_vector)
    push!(other_prices_list, price_vector)
end;

In [53]:
lower[end] = -100.0
upper[end] = 0.0

0.0

In [54]:
@time ll_fnc(init_params,prices,coins,copay,df)

  0.018084 seconds (92.94 k allocations: 17.723 MiB)


72385.4344119039

In [55]:
length(δ)

48

In [56]:
size(df,1)

18582

In [ ]:
@time res1 = optimize_function(init_params,prices,coins,copay,df,lower,upper, ll_fnc)

In [40]:
δ_full = res1.minimizer[1:end-1]

40-element Vector{Float64}:
 14.267161612455556
 16.352143336649327
 18.26638116616527
 14.39232475944487
 18.466866693835314
 16.30693790131152
 17.230141102132247
 16.154231263661156
 17.122353866428675
 17.068499750807423
 16.79200097608911
 15.748766156790944
 15.187886945875556
 16.68901012989665
 17.826944273728408
 14.015847182150777
 16.40067037927677
 14.764564217382858
 16.241242642171937
 17.882274919228976
 15.945296058855686
 16.14747448189096
 15.365773903493476
 14.854948281285674
 13.986859647928195
 18.01116036954902
 15.104890027023187
 13.23308784827952
 14.982287702377619
 13.233087844998217
 16.61323181973874
 16.514938472622063
 15.214089315822527
 15.584463105037711
 14.310646726250429
 13.82779495587364
 14.411742842014032
 13.505021563519634
 13.756335989283768
 13.792703632855941

In [41]:
α_full = res1.minimizer[end]

-9.942298848989804e-25

In [42]:
estimates_dict = Dict{String, Any}()

Dict{String, Any}()

In [43]:
estimates_dict["Year"] = "2017"
for (i,j) in enumerate(product_dict)
    #println(j[1], ": ",δ_full[j[2]])
    estimates_dict[j[1]] = δ_full[j[2]]
end
estimates_dict["α"] = α_full

-9.942298848989804e-25

In [28]:
estimated_df = DataFrame(estimates_dict)

Row,Adderall,CONCERTA,Daytrana,"Dextroamphetamine Saccharate, Amphetamine Aspartate, Dextroamphetamine Sulfate and Amphetamine Sulfate",Dextroamphetamine Sulfate,"Dextroamphetamine saccharate, amphetamine aspartate monohydrate, dextroamphetamine sulfate, amphetamine sulfate",FENTANYL,Focalin,Hydrocodone Bitartrate And Acetaminophen,Hydrocodone Bitartrate and Acetaminophen,Hydrocodone Bitartrate and Homatropine Methylbromide,Hydromet,Hydromorphone Hydrochloride,METHADONE HYDROCHLORIDE,MORPHINE SULFATE,Methadone Hydrochloride,Methylphenidate Hydrochloride,Morphine Sulfate,OXYCODONE AND ACETAMINOPHEN,OXYCODONE HYDROCHLORIDE,OxyContin,Oxycodone Hydrochloride,Oxycodone and Acetaminophen,Oxymorphone Hydrochloride,PERCOCET,Ritalin,Vyvanse,Year,oxycodone hydrochloride,α
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Float64,Float64
1,17.5796,15.3725,15.3498,18.3893,15.3265,15.6023,16.1971,16.9886,13.5229,16.6042,15.2013,15.479,17.269,16.9864,16.5572,14.1027,17.2724,16.8474,17.9409,18.4041,17.9235,15.7122,16.6738,13.7742,14.5727,15.8326,18.3948,2012,14.4948,-8.92394e-43


In [44]:
CSV.write("est_coefs_2017_v2.csv", estimated_df)

"est_coefs_2017_v2.csv"

In [385]:
column_names = ["Column_A", "Column_B", "Column_C", "Column_D", "Column_E"]
estimated_df = DataFrame([Symbol(name) => Vector{Any}() for name in column_names])

Row,Column_A,Column_B,Column_C,Column_D,Column_E
,Any,Any,Any,Any,Any


In [395]:
my_dict = Dict("Name" => ["Alice", "Bob", "Charlie"],
               "Age" => [25, 30, 22],
               "City" => ["New York", "London", "Tokyo"])

# Convert the dictionary to a DataFrame
df = DataFrame(my_dict)

# Display the DataFrame
println(df)

3×3 DataFrame
 Row │ Age    City      Name    
     │ Int64  String    String  
─────┼──────────────────────────
   1 │    25  New York  Alice
   2 │    30  London    Bob
   3 │    22  Tokyo     Charlie


In [265]:
# column_types = Dict(
#     :PROV_KEY => String,
#     :PROV_CW_KEY => String,
#     :FACILITY_NAME => String,
#     :PROV_TYPE => String,
#     :PROV_TYPE_ORIG => String,
#     :PROV_CLINIC_STATE => String,
#     :NH_COUNTY_CODE => String,
#     :NDC => String,
#     :COVERAGE_CLASS => String,
#     :FROM_YEAR => String,
#     :CLAIM_ID_KEY => String,
#     :SV_LINE => String,
#     :SV_STAT => String,
#     :AGE => String,
#     :SEX => String,
#     :MEMBER_COUNTY => String,
#     :MEMBER_STATE => String,
#     :PHARMACY_COUNTY => String,
#     :BILL_PROV_CW_KEY => String,
#     :QTY => String,
#     :AMT_BILLED => String,
#     :AMT_PAID => String,
#     :AMT_DEDUCT => String,
#     :AMT_COINS => String,
#     :AMT_COPAY => String,
#     :NDC_PROD_NAME => String,
#     :BRAND_STATUS => String,
#     :RX_DAYS_SUPPLY => String,
#     :RX_INGR_COST => String,
#     :RX_DISP_FEE => String,
#     :RX_DAW => String,
#     :RX_REFILLS=> Int, 
    
# )

# for (i,j) in enumerate(column_types)
#     df[!,i] = convert(Vector{j[2]}, df[!,i])
# end